In [0]:
jdbc_url = (
    "jdbc:postgresql://azuremaniezzijp.postgres.database.azure.com:5432/vendas"
    "?sslmode=require"
)

# OBSERVAÇÃO : NA URL SEMPRE LEMBRE DE COLOCAR O BANCO DE DADOS NO FINAL DA URL

connection_properties = {
    "user": "PgAdminJp",
    "password": "***",
    "driver": "org.postgresql.Driver"
}




In [0]:

df_clientes = spark.read.jdbc(
    url=jdbc_url,
    table="public.clientes",
    properties=connection_properties
)

display(df_clientes)

bronze_path = "dbfs:/mnt/datalake/bronze/postgres/vendas/clientes"
(
    df_clientes
    .write
    .format("delta")
    .mode("overwrite")  # Bronze pode ser recriado
    .save(bronze_path)
)


# df_produtos = spark.read.jdbc(
#     url=jdbc_url,
#     table="public.produtos",
#     properties=connection_properties
# )

# display(df_produtos)

# bronze_path = "dbfs:/mnt/datalake/bronze/postgres/vendas/produtos"
# (
#     df_produtos.write
#     .format("delta")
#     .mode("overwrite")
#     .save(bronze_path)
# )
df_pedido = spark.read.jdbc(
    url=jdbc_url,
    table="public.pedido",
    properties=connection_properties
)

display(df_pedido)

bronze_path = "dbfs:/mnt/datalake/bronze/postgres/vendas/pedido"
(
    df_pedido.write                  
    .format("delta")
    .mode("overwrite")
    .save(bronze_path)
)
df_itens_pedido = spark.read.jdbc(
    url=jdbc_url,
    table="public.itens_pedido",
    properties=connection_properties
)

display(df_itens_pedido)

bronze_path = "dbfs:/mnt/datalake/bronze/postgres/vendas/itens_pedido"
(
    df_itens_pedido.write
    .format("delta")
    .mode("overwrite")
    .save(bronze_path)
)

In [0]:
dbutils.fs.mkdirs("dbfs:/mnt/datalake/bronze/postgres/vendas/clientes")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/bronze/postgres/vendas/produtos")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/bronze/postgres/vendas/pedidos")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/bronze/postgres/vendas/itens_pedido")

dbutils.fs.mkdirs("dbfs:/mnt/datalake/silver/vendas/clientes")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/silver/vendas/produtos")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/silver/vendas/pedidos")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/silver/vendas/itens_pedido")

dbutils.fs.mkdirs("dbfs:/mnt/datalake/gold/vendas/fato_vendas")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/gold/vendas/dim_clientes")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/gold/vendas/dim_produtos")


In [0]:
dbutils.fs.ls("dbfs:/mnt")
dbutils.fs.ls("dbfs:/mnt/datalake")
dbutils.fs.ls("dbfs:/mnt/datalake/bronze")
dbutils.fs.ls("dbfs:/mnt/datalake/silver")
dbutils.fs.ls("dbfs:/mnt/datalake/gold")


